# Импорты

In [58]:
import os
import sys

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [59]:
# Импортируем функции предобработки из utils
from utils import preprocess_functions as ppf
from utils import metrics as mtc

In [60]:
import re


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import joblib
import itertools

# Загружаем разделенные данные

In [61]:
reference = pd.read_csv("../../data/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")
test = pd.read_csv("../../data/splited/test.csv")

In [62]:
reference.head(1)

,id,name,region
0,69,Кировска,Мурманская область


In [63]:
train.head(1)

,name,school_id
0,"Москва, ГБУ МАФКК школа ""Мечта""",102


# Обработка reference.region

In [64]:
reference.region = reference.region.apply(ppf.simple_preprocess_text).str.lower()

In [65]:
region_dict = joblib.load("../../resources/general/region_dict.joblib")

# joblib.dump(region_dict, "../../resources/general/region_dict.joblib")

In [66]:
for reference_region in reference.region.sort_values().unique():
    if reference_region not in region_dict:
        for region in region_dict:
            if reference_region in region_dict[region]:
                reference.region = reference.region.str.replace(
                    f"{reference_region}", f"{region}"
                )
                print(f"{reference_region} -> {region}")

набережные челны -> республика татарстан
северодвинск -> архангельская область


In [67]:
for reference_region in reference.region.sort_values().unique():
    if reference_region not in region_dict:
        print(f"Unknown region: {reference_region}")

Unknown region: республика саха
Unknown region: республика чувашия
Unknown region: хмао югра
Unknown region: ямало ненецкий ао


In [68]:
reference.region = reference.region.str.replace(
    "республика саха",
    "республика саха якутия",
)
reference.region = reference.region.str.replace(
    "республика чувашия",
    "чувашская республика",
)
reference.region = reference.region.str.replace(
    "хмао югра",
    "ханты мансийский автономный округ",
)
reference.region = reference.region.str.replace(
    "ямало ненецкий ао",
    "ямало ненецкий автономный округ",
)

In [69]:
reference.head()

,id,name,region
0,69,Кировска,мурманская область
1,70,Клин спортивный,московская область
2,71,КО СШ по ЗВС,калининградская область
3,8,Айсберг,свердловская область
4,73,Команда Дмитриева,челябинская область


In [70]:
reference.to_csv("../../data/preprocessed/general/reference.csv", index=False)

# Простая обработка текста

In [71]:
reference["new"] = reference.name.apply(ppf.simple_preprocess_text).str.lower()
reference.new = reference.new.apply(ppf.replace_numbers_with_text)

In [72]:
reference.loc[[187, 51], :]

,id,name,region,new
187,206,СШОР No 1,самарская область,сшор один
51,53,ДЮСШ No 1,тюменская область,дюсш один


# Раширенная обработка текста: работа с аббревиатурами

## Удаление всех аббревиатур

Формируем blacklist и whitelist аббревиатур, для этого не обязательно иметь аббревиатуры в верхнем регистре. Аббериватуры из blacklist будут удалены.

In [73]:
abbr_dict = joblib.load("../../resources/general/abbreviations_dict.joblib")
# joblib.dump(abbr_dict, "../../resources/general/abbreviations_dict.joblib")


In [74]:
abbr_dict = {
    "хмао": "ханты мансийский автономный округ",
    "югра": "",
    "динамо": "динамо",
    "история": "история",
    "олимпия": "олимпия",
    "сияние": "сияние",
    "пгуфксит": "поволжский государственный университет физической культуры спорта и туризма",
    "сит": "спорта и туризма",
    "ледовая": "ледовая",
    "шфкнк": "школа фигурного катания на коньках",
    "цпфких": "центр подготовки по фигурному катанию на коньках и хоккею",
    "восточная": "восточная",
    "физ": "физкультуры и здоровья",
    "надежды": "надежды",
    "фкких": "фигурному катанию на коньках и хоккею",
    "цфиз": "центр физкультуры и здоровья",
    "аго": "ангарский городской округ",
    "алвс": "академия ледовых видов спорта",
    "ано": "автономная некоммерческая организация",
    "анфсо": "автономная некоммерческая физкультурно спортивная организация",
    "ао": "автономный округ",
    "ау": "автономное учреждение",
    "афк": "академия фигурного катания",
    "барс": "березники арена спорт",
    "бу": "бюджетное учреждение",
    "вдв": "воздушно десантные войска",
    "витязь": "витязь",
    "во": "василеостровского района",
    "всшор": "всеволжская спортивная школа олимпийского резерва",
    "вфсо": "всероссийское физкультурно спортивное общество",
    "гау": "государственное автономное учреждение",
    "гбоу": "государственное бюджетное образовательное учреждение",
    "гбпоу": "государственное бюджетное профессиональное образовательное учреждение",
    "гбу": "государственное бюджетное учреждение",
    "глайд": "глайд",
    "гоау": "государственное областное автономное учреждение",
    "гу": "государственное учреждение",
    "до": "дополнительного образования",
    "дод": "дополнительного образования детей",
    "дс": "дворец спорта",
    "дюсош": "детско юношеская спортивно оздоровительная школа",
    "дюсш": "детско юношеская спортивная школа",
    "дюц": "детско юношеский центр",
    "звс": "зимние виды спорта",
    "золотые": "золотые",
    "ио": "иркутская область",
    "ип": "индивидуальный предприниматель",
    "иро": "иркутское региональное отделение",
    "кгбу": "краевое государственное бюджетное учреждение",
    "кк": "краснодарского края",
    "ко": [
        "калужская область",
        "костромская область",
        "кировская область",
        "калининградская область",
    ],
    "когау": "кировское областное государственное автономное учреждение",
    "кроо": "краевая региональная общественная организация",
    "кфк": "клуб фигурного катания",
    "кфкнк": "клуб фигурного катания на коньках",
    "лвс": "ледовые виды спорта",
    "лд": "ледовый дворец",
    "лдс": "ледовый дворец спорта",
    "ло": "ленинградская область",
    "лсвс": "ледовым видам спорта",
    "лск": "ледовый спортивный комплекс",
    "лц": "ледовый центр",
    "ма": "московская академия",
    "маоу": "муниципальное автономное образовательное учреждение",
    "мау": "муниципальное автономное учреждение",
    "маудо": "муниципальное автономное учреждение дополнительного образования",
    "маусш": "муниципальное автономное учреждение спортивная школа",
    "мафк": "московская академия фигурного катания на коньках",
    "мафкк": "московская академия фигурного катания на коньках",
    "мафсу": "муниципальное автономное физкультурно спортивное учреждение",
    "мбоу": "муниципальное бюджетное образовательное учреждение",
    "мбу": "муниципальное бюджетное учреждение",
    "мбудо": "муниципальное бюджетное учреждение дополнительного образования",
    "мбус": "муниципальное бюджетное учреждение спорта",
    "мбфсу": "муниципальное бюджетное физкультурно спортивное учреждение",
    "мдмис": "министерство по делам молодёжи и спорту",
    "мку": "муниципальное казенное учреждение",
    "мо": [
        "муниципального образования",
        "московской области",
        "мурманской области",
        "министерства обороны",
    ],
    "мо рф": "министерства обороны россии",
    "мо россии": "министерства обороны россии",
    "мосгорспорт": "московская дирекция по развитию массового спорта",
    "моу": "муниципальное образовательное учреждение",
    "му": "муниципальное учреждение",
    "наши": "наши",
    "нао": "ненецкий автономный округ",
    "нлфк": "национальная лига фигурного катания",
    "но": "нижегородская область",
    "нп": "некоммерческое партнерство",
    "огау": "областное государственное автономное учреждение",
    "ого": "общественно государственное объединение",
    "оксшор": "областная комплексная спортивная школа олимпийского резерва",
    "олимп": "олимп",
    "обл": "область",
    "оо": ["общественная организация", "общественное объединение"],
    "ооо": "общество с ограниченной ответственностью",
    "ооффк": "общественная организация федерация фигурного катания",
    "орк": "олимпийского резерва, комплексная",
    "оур": "училище техникум олимпийского резерва",
    "пбму": "первоуральское муниципальное бюджетное учреждение",
    "пмбу": "первоуральское муниципальное бюджетное учреждение",
    "по": "псковской области",
    "проо": "псковская региональная общественная организация",
    "рб": ["республика башкортостан", "республика беларусь"],
    "респ": "республика",
    "рк": [
        "республика карелия",
        "республика калмыкия",
        "республика коми",
        "республика крым",
        "республика казахстан",
    ],
    "рм": "республика мордовия",
    "рмэ": "республика марий эл",
    "ро": ["рязанская область", "региональное объединение", "ростовская область"],
    "роо": "региональная общественная организация",
    "роффкк": "ростовская областная федерация фигурного катания на коньках",
    "рс": "республика саха якутия",
    "рсоо": "региональная спортивная общественная организация",
    "рссш": "республики саха якутия спортивная школа",
    "рсшор": "республиканская спортивная школа олимпийского резерва",
    "рся": "республика саха якутия",
    "рт": "республика татарстан",
    "рф": "российская федерация",
    "рфсоо": "региональная физкультурно спортивная общественная организация",
    "рцсп": "региональный центр спортивной подготовки",
    "рцспзвс": "региональный центр спортивной подготовки по зимним видам спорта",
    "саха": "республики саха",
    "сбс": "сбс",
    "ск": "спортивный клуб",
    "ска": "спортивный клуб армии",
    "скфк": "спортивный клуб фигурного катания",
    "со": ["самарская область", "свердловская область", "смоленская область"],
    "согбу": "смоленское областное государственное бюджетное учреждение",
    "согбоу": "смоленское областное государственное бюджетное образовательное учреждение",
    "спб": "санкт петербург",
    "стартайс": "cтартайс",
    "стц": "спортивно тренировочный центр",
    "сфк": "секция фигурного катания",
    "сффк": "спортивная федерация фигурного катания на коньках",
    "сш": "спортивная школа",
    "сшор": "спортивная школа олимпийского резерва",
    "сшфкк": "спортивная школа фигурного катания",
    "тим": "тим",
    "титул": "титул",
    "то": "тульской области",
    "топ": "топ",
    "узор": "узор",
    "уор": "училище техникум олимпийского резерва",
    "ур": "удмуртской республики",
    "уффк": "удмуртская федерация фигурного катания",
    "фау": "федеральное автономное учреждение",
    "фк": "фигурного катания",
    "фкис": "физической культуры и спорта",
    "фкишт": "фигурному катанию на коньках и шорт треку",
    "фккишт": "фигурному катанию на коньках и шорт треку",
    "фкк": "фигурного катания на коньках",
    "фксиз": "физической культуры спорта и здоровья",
    "фок": "физкультурно оздоровительный комплекс",
    "фс": "физкультурно спортивное",
    "фск": "физкультурно спортивный комплекс",
    "фсо": "физкультурно спортивное объединение",
    "ффк": "федерация фигурного катания",
    "ффкк": "федерация фигурного катания на коньках",
    "ффккво": "федерация фигурного катания на коньках владимирской области",
    "ффккрк": "федерация фигурного катания на коньках республики коми",
    "ффккск": "федерация фигурного катания на коньках ставропольского края",
    "ффкпо": "федерация фигурного катания на коньках псковской области",
    "хк": "хоккейный клуб",
    "хшфк": "спортивный клуб по хоккею с шайбой и фигурному катанию",
    "цзвс": "центр зимних видов спорта",
    "цоп": "центр олимпийской подготовки",
    "цплвс": "центр по ледовым видам спорта",
    "цпср": "центр подготовки спортивного резерва",
    "црс": "центр развития спорта",
    "црфк": "центр развития фигурного катания",
    "цска": "центральный спортивный клуб армии",
    "цсп": "центр спортивной подготовки",
    "цфк": "центр физической культуры",
    "цфксиз": "центр физической культуры спорта и здоровья",
    "цфкиз": "центр физической культуры и здоровья",
    "цфкис": "центр физической культуры и спорта",
    "чоу": "частное образовательное учреждение",
    "чр": ["чувашская республика", "чеченская республика"],
    "чу": "частное учреждение",
    "чудо": "частное учреждение дополнительного образования",
    "шфк": "школа фигурного катания",
    "юнармейцы": "юнармецны",
    "янао": "ямало ненецкий автономный округ",
}


Расшифровываем все аббревиатуры

In [75]:
def preprocess_text(
    name,
    abbreviation_dict,
    output_list=False,
    unknown_answer=False,
    remove_unknown_abbr=False,
    remove_all_abbr=False,
):
    two_letter_prepositions = [
        " в ",
        " во ",
        " до ",
        " из ",
        " на ",
        " по ",
        " о ",
        " об ",
        " обо ",
        " у ",
    ]

    symbols = [" no", " NO", " No", "номер"]

    # Удаляем служебные символы (перенос строки, табуляция и т.д.)
    name = re.sub(r"[\n\t\r]", " ", name)

    # Создаем регулярное выражение для предлогов
    prepositions_pattern = (
        r"\b(?:" + "".join(two_letter_prepositions) + "".join(symbols) + r")\b"
    )

    # Заменяем все предлоги на пробем (предварительное решение вместо трудоемкого удаления стоп-слов)
    name = re.sub(prepositions_pattern, " ", name)

    # Удаление пунктуации
    name = re.sub(r"[^\w\s]", " ", name)

    # Удаление отдельных букв
    name = re.sub(r"\b[А-ЯЁа-яё]\b", " ", name)

    # Удаление букв ё
    name = re.sub(r"[Ёё]", "е", name)

    unknown_abbr = []
    # Находим аббревиатуры большими буквами и приводим их к нижнему регистру
    # Надо уточнить поиск неизвестных.
    # А если в конце аббревиатуры прописная буква?
    uppercase_abbreviations = re.findall(r"\b[А-ЯЁ]+[а-яё]*+[А-ЯЁ]+\b", name)
    for abbr in uppercase_abbreviations:
        abbr = abbr.lower()
        if abbr not in abbreviation_dict:
            unknown_abbr.append(abbr.upper())
            if remove_unknown_abbr:
                name = re.sub(r"\b" + re.escape(abbr.upper()) + r"\b", " ", name)

    # Удаление лишних пробелов
    name = re.sub(r"\s+", " ", name)

    # Удаление пробелов в начале и в конце
    name = name.strip()

    possible_replacements = []
    parts = name.lower().split()

    for part in parts:
        if part in abbreviation_dict:
            if not remove_all_abbr:
                replacements = abbreviation_dict[part]
                if isinstance(replacements, str):
                    replacements = [replacements]
                elif not output_list:
                    replacements = [""]
                possible_replacements.append(replacements)
            else:
                pass
        else:
            possible_replacements.append([part])

    # Генерируем все возможные комбинации
    all_combinations = list(itertools.product(*possible_replacements))

    # Формируем итоговые наименования
    final_phrases = [" ".join(combination).strip() for combination in all_combinations]

    if not output_list:
        final_phrases = final_phrases[0]

    if unknown_answer:
        return list(set(unknown_abbr))

    return final_phrases


In [76]:
reference["extended_process"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.extended_process = reference.extended_process.apply(
    ppf.replace_numbers_with_text
)
reference.extended_process = reference.extended_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, True)
)


In [77]:
reference.loc[[187, 51], :]

,id,name,region,new,extended_process
187,206,СШОР No 1,самарская область,сшор один,один
51,53,ДЮСШ No 1,тюменская область,дюсш один,один


## Расшифровка аббревиатур, удаление неизвестных аббревиатур

In [78]:
reference["extended2_process"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.extended2_process = reference.extended2_process.apply(
    ppf.replace_numbers_with_text
)
reference.extended2_process = reference.extended2_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)


In [79]:
reference.loc[[187, 51], :]

,id,name,region,new,extended_process,extended2_process
187,206,СШОР No 1,самарская область,сшор один,один,спортивная школа олимпийского резерва один
51,53,ДЮСШ No 1,тюменская область,дюсш один,один,детско юношеская спортивная школа один


# Работа с регионами

## Расшифровка аббревиатур, удаление неизвестных аббревиатур

In [80]:
train["region_process"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.region_process = train.region_process.apply(ppf.replace_numbers_with_text)
train.region_process = train.region_process.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)


In [82]:
train["region"] = train.region_process.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.region_process = train.region_process.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)


In [83]:
train.head(15)

,name,school_id,region_process,region
0,"Москва, ГБУ МАФКК школа ""Мечта""",102,государственное бюджетное учреждение московска...,москва
1,"Калужская область, СШ ""Космос""",78,спортивная школа космос,калужская область
2,"МО 7-й округ, ООО ""Династия""",49,семь округ общество с ограниченной ответственн...,None
3,АНФСО СКФК Спартак,164,автономная некоммерческая физкультурно спортив...,None
4,"Челябинская область, ИП Дмитриев А.А.",73,индивидуальный предприниматель дмитриев,челябинская область
5,"Санкт-Петербург, СПб ГБУ СШОР ""Звездный лед""""",62,государственное бюджетное учреждение спортивна...,санкт петербург
6,"ООО ""Вилеса Айс""",30,общество с ограниченной ответственностью вилес...,None
7,"НП ""КФК ""Динамо-Санкт-Петербург""",277,некоммерческое партнерство клуб фигурного ката...,санкт петербург
8,"Пензенская область, МБУ ДО ""СШ по зимним видам...",304,муниципальное бюджетное учреждение дополнитель...,пензенская область
9,"Владимирская область, МБУ ДО ""СШ №8""",59,муниципальное бюджетное учреждение дополнитель...,владимирская область
